In [1]:
import pandas as pd
import json

# Download & Curate all Datasets

In [4]:
# Bible Dataset

with open('data/Bible/bible_eng.json', encoding="utf8") as file_en:
    bible_en = json.load(file_en)
    

with open('data/Bible/bible_rw.json', encoding="utf8") as file_rw:
    bible_rw = json.load(file_rw)
    
    
def parse_bible(version):
    lines = []
    index = []
    iters = 0
    err = []
    for test_index, testament in enumerate(version['bible']['testament']):
        for book_index , book in enumerate(testament['book']):
            for chapt_index, chapter in enumerate(book['chapter']):
                try:
                    for verse_index, verse in enumerate(chapter['verse']):
                        lines.append({'Testament':test_index, 'Book':book_index, 'Chapter':chapt_index, 'Line':verse_index, 'Verse': verse['#text']})
                        index.append(iters)
                        iters += 1
                except TypeError:
                    err.append(iters)
                    iters += 1
    return lines, index, err
        
en, index_en, e_en = parse_bible(bible_en)
rw, index_rw, e_rw = parse_bible(bible_rw)
    
bible_df_en = pd.DataFrame(en)

bible_df_rw = pd.DataFrame(rw)

bible_merge = bible_df_en.merge(bible_df_rw, how='inner', left_on=["Testament", "Book", "Chapter","Line"], right_on=["Testament", "Book", "Chapter","Line"])

bible = pd.DataFrame({'rw':bible_merge.Verse_y,'en':bible_merge.Verse_x})
bible

,rw,en
0,Mbere na mbere Imana yaremye ijuru n'isi.,In the beginning God created the heaven and th...
1,"Isi yari itagira ishusho, yariho ubusa busa, u...","And the earth was without form, and void; and ..."
2,"Imana iravuga iti “Habeho umucyo”, umucyo ubaho.","And God said, Let there be light: and there wa..."
3,"Imana ibona umucyo ko ari mwiza, Imana itanduk...","And God saw the light, that it was good: and G..."
4,"Imana yita umucyo amanywa, umwijima iwita ijor...","And God called the light Day, and the darkness..."
...,...,...
30868,Umwuka n'umugeni barahamagara bati “Ngwino!” K...,"And the Spirit and the bride say, Come. And le..."
30869,Uwumva wese amagambo y'ubuhanuzi bw'iki gitabo...,For I testify unto every man that heareth the ...
30870,Kandi nihagira umuntu ukura ku magambo y'igita...,And if any man shall take away from the words ...
30871,"Uhamya ibyo aravuga ati “Yee, ndaza vuba.”Amen...","He which testifieth these things saith, Surely..."


In [5]:
#Quran Dataset

quran_eng = pd.read_csv('data/Quran/quran_eng.csv')
quran_rw = pd.read_csv('data/Quran/quran_rw.csv')

quran = pd.DataFrame({'rw': quran_rw.translation,'en': quran_eng.translation})
quran

,rw,en
0,"1. Ku izina rya Allah[1], Nyirimpuhwe, Nyirimb...","(1) In the name of Allāh,[2] the Entirely Merc..."
1,"2. Ishimwe n’ikuzo byose bikwiye Allah, Nyagas...","(2) [All] praise is [due] to Allāh, Lord[4] of..."
2,"3. Nyirimpuhwe, Nyirimbabazi","(3) The Entirely Merciful, the Especially Merc..."
3,4. Umwami w’ikirenga wo ku munsi w’imperuka.,(4) Sovereign of the Day of Recompense.[5]
4,"5. Ni wowe (wenyine) dusenga, kandi ni wowe (w...",(5) It is You we worship and You we ask for help.
...,...,...
6231,"2. “Umwami w’abantu,”","(2) The Sovereign of mankind,"
6232,"3. “Imana y’abantu,”","(3) The God of mankind,"
6233,4. “Ngo andinde ububi bwa (shitani) yoshya guk...,(4) From the evil of the retreating whisperer[...
6234,5. “Yoshya imitima y’abantu gukora ibibi; ”,(5) Who whispers [evil] into the breasts of ma...


In [16]:
#Digital Umuganda

corpus_1 = pd.read_csv('data/umuganda/corpus_1.tsv', encoding = "ISO-8859-1", sep='\t')
corpus_2 = pd.read_csv('data/umuganda/corpus_2.tsv', encoding = "ISO-8859-1", sep='\t')

corpus = corpus_1.append(corpus_2, ignore_index=True)

# Merge Datasets

In [118]:
dataset = bible.append(quran, ignore_index=True)

In [119]:
dataset

,rw,en
0,Mbere na mbere Imana yaremye ijuru n'isi.,In the beginning God created the heaven and th...
1,"Isi yari itagira ishusho, yariho ubusa busa, u...","And the earth was without form, and void; and ..."
2,"Imana iravuga iti “Habeho umucyo”, umucyo ubaho.","And God said, Let there be light: and there wa..."
3,"Imana ibona umucyo ko ari mwiza, Imana itanduk...","And God saw the light, that it was good: and G..."
4,"Imana yita umucyo amanywa, umwijima iwita ijor...","And God called the light Day, and the darkness..."
...,...,...
37104,"2. “Umwami w’abantu,”","(2) The Sovereign of mankind,"
37105,"3. “Imana y’abantu,”","(3) The God of mankind,"
37106,4. “Ngo andinde ububi bwa (shitani) yoshya guk...,(4) From the evil of the retreating whisperer[...
37107,5. “Yoshya imitima y’abantu gukora ibibi; ”,(5) Who whispers [evil] into the breasts of ma...
